# CIA Triad and Cryptographic Methods

## Contents
1. CIA Triad Definition
2. Confidentiality
3. Integrity
4. Availability
5. Practical Examples

## Prerequisites:

```bash
pip install cryptography hmac
```

In [ ]:
from cryptography.hazmat.primitives import hashes, hmac, serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.backends import default_backend
from cryptography.fernet import Fernet
import os
import hashlib
import time
import secrets
import hmac as hmac_lib

print("✓ Libraries loaded")

## 1. CIA Triad Definition

**CIA Triad** = Three fundamental principles of information security

### Components:

| Principle | Definition | Goal |
|-----------|------------|------|
| **Confidentiality** | Data accessible only to authorized parties | Prevent unauthorized disclosure |
| **Integrity** | Data remains accurate and unmodified | Prevent unauthorized modification |
| **Availability** | Data accessible when needed | Ensure system uptime and access |

### Security Balance:
- All three must be balanced for effective security
- Strengthening one may affect others
- Different scenarios prioritize different aspects

## 2. Confidentiality

**Definition:** Ensuring information is disclosed only to authorized parties

### Cryptographic Methods:
- **Encryption** (symmetric/asymmetric)
- **Access controls** with authentication
- **Key management**
- **Secure channels** (TLS/SSL)

### How Cryptography Helps:
- Encrypts data at rest and in transit
- Only authorized parties with keys can decrypt
- Protects against eavesdropping and theft

### Example 1: File Encryption (AES)

In [ ]:
print("CONFIDENTIALITY EXAMPLE 1: File Encryption")
print("=" * 70)
print("Use Case: Protecting sensitive files on disk\n")

# Sensitive data
confidential_data = "Patient medical records: John Doe, Diagnosis: Confidential"

# Generate encryption key
key = Fernet.generate_key()
cipher = Fernet(key)

# Encrypt
encrypted = cipher.encrypt(confidential_data.encode())

print(f"Original: {confidential_data}")
print(f"\nEncrypted: {encrypted[:60]}...")
print("\n✓ Without key, data is unreadable")

# Decrypt (authorized access)
decrypted = cipher.decrypt(encrypted).decode()
print(f"\nWith key: {decrypted}")
print("\n✓ Confidentiality maintained through encryption")

### Example 2: HTTPS/TLS Communication

In [ ]:
print("CONFIDENTIALITY EXAMPLE 2: Secure Communication")
print("=" * 70)
print("Use Case: Online banking, e-commerce\n")

# Simulate asymmetric encryption for key exchange
private_key = rsa.generate_private_key(
    public_exponent=65537, key_size=2048, backend=default_backend()
)
public_key = private_key.public_key()

# Sensitive data (e.g., credit card)
credit_card = "4532-1234-5678-9010"

# Encrypt with public key
encrypted = public_key.encrypt(
    credit_card.encode(),
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None
    ),
)

print(f"Credit card: {credit_card}")
print(f"Encrypted (first 60 bytes): {encrypted[:60].hex()}...")
print("\n✓ Data encrypted in transit (HTTPS)")
print("✓ Eavesdroppers cannot read the data")
print("✓ Only server with private key can decrypt")

### Example 3: Database Encryption

In [ ]:
print("CONFIDENTIALITY EXAMPLE 3: Database Column Encryption")
print("=" * 70)
print("Use Case: Protecting PII (Personally Identifiable Information)\n")

# Simulate database records
key = Fernet.generate_key()
cipher = Fernet(key)

users = [
    {"id": 1, "name": "Alice", "ssn": "123-45-6789"},
    {"id": 2, "name": "Bob", "ssn": "987-65-4321"},
]

# Encrypt sensitive fields
for user in users:
    user["ssn_encrypted"] = cipher.encrypt(user["ssn"].encode())
    del user["ssn"]  # Remove plaintext

print("Encrypted Database Records:")
for user in users:
    print(f"  ID: {user['id']}, Name: {user['name']}")
    print(f"  SSN: {user['ssn_encrypted'][:50]}...")

print("\n✓ SSN encrypted at rest")
print("✓ Database breach doesn't expose sensitive data")
print("✓ Encryption key stored separately (HSM/Key vault)")

## 3. Integrity

**Definition:** Ensuring information remains accurate and unmodified

### Cryptographic Methods:
- **Hash functions** (SHA-256, SHA-3)
- **Digital signatures** (RSA, ECDSA)
- **HMAC** (Hash-based Message Authentication Code)
- **Checksums** and MACs

### How Cryptography Helps:
- Detects any modification to data
- Verifies data authenticity
- Prevents tampering
- One-way functions make reversal impossible

### Example 1: File Integrity Check (Hash)

In [ ]:
print("INTEGRITY EXAMPLE 1: File Download Verification")
print("=" * 70)
print("Use Case: Software downloads, OS images\n")

# Original file content
original_file = b"Ubuntu-22.04-desktop-amd64.iso (simulated content)"

# Calculate hash (publisher provides this)
digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest.update(original_file)
original_hash = digest.finalize()

print("File: ubuntu-22.04.iso")
print(f"Published SHA-256: {original_hash.hex()}")

# User downloads and verifies
downloaded_file = original_file
digest2 = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest2.update(downloaded_file)
downloaded_hash = digest2.finalize()

print(f"\nCalculated SHA-256: {downloaded_hash.hex()}")
print(f"\n✓ Match: {original_hash == downloaded_hash}")
print("✓ File not corrupted or tampered with")

# Simulate tampering
tampered_file = b"Ubuntu-22.04-desktop-amd64.iso (MODIFIED MALWARE)"
digest3 = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest3.update(tampered_file)
tampered_hash = digest3.finalize()

print(f"\nTampered SHA-256: {tampered_hash.hex()}")
print(f"Match: {original_hash == tampered_hash}")
print("✗ Integrity check failed - file modified!")

### Example 2: Digital Signatures

In [ ]:
print("INTEGRITY EXAMPLE 2: Software Code Signing")
print("=" * 70)
print("Use Case: Windows drivers, mobile apps\n")

# Developer's key pair
private_key = rsa.generate_private_key(
    public_exponent=65537, key_size=2048, backend=default_backend()
)
public_key = private_key.public_key()

# Software code
software_code = b"Application.exe binary data"

# Developer signs the code
signature = private_key.sign(
    software_code,
    padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
    hashes.SHA256(),
)

print("Software: Application.exe")
print(f"Signature (first 40 bytes): {signature[:40].hex()}...")

# User verifies signature
try:
    public_key.verify(
        signature,
        software_code,
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256(),
    )
    print("\n✓ Signature valid")
    print("✓ Code verified from trusted developer")
    print("✓ Code not modified since signing")
except Exception:
    print("\n✗ Signature invalid - code tampered!")

# Test with modified code
modified_code = b"Application.exe binary data MALWARE"
try:
    public_key.verify(
        signature,
        modified_code,
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256(),
    )
    print("\nSignature valid")
except Exception:
    print("\n✗ Modified code: Signature verification failed!")
    print("✗ Integrity compromised")

### Example 3: HMAC for Message Authentication

In [ ]:
print("INTEGRITY EXAMPLE 3: API Message Authentication")
print("=" * 70)
print("Use Case: REST APIs, webhook validation\n")

# Shared secret key between client and server
secret_key = os.urandom(32)

# API request data
api_request = b'{"user": "alice", "action": "transfer", "amount": 1000}'

# Client creates HMAC
h = hmac.HMAC(secret_key, hashes.SHA256(), backend=default_backend())
h.update(api_request)
mac = h.finalize()

print(f"API Request: {api_request.decode()}")
print(f"HMAC: {mac.hex()}")

# Server verifies HMAC
h2 = hmac.HMAC(secret_key, hashes.SHA256(), backend=default_backend())
h2.update(api_request)

try:
    h2.verify(mac)
    print("\n✓ HMAC verified")
    print("✓ Message not modified in transit")
    print("✓ Request from authenticated client")
except Exception:
    print("\n✗ HMAC verification failed!")

# Simulate MITM attack
tampered_request = b'{"user": "alice", "action": "transfer", "amount": 9999}'
h3 = hmac.HMAC(secret_key, hashes.SHA256(), backend=default_backend())
h3.update(tampered_request)

try:
    h3.verify(mac)  # Using original MAC with modified data
    print("\nHMAC verified")
except Exception:
    print("\n✗ Tampered request: HMAC mismatch!")
    print("✗ Attacker modified amount - detected!")

## 4. Availability

**Definition:** Ensuring authorized users can access information when needed

### Cryptographic Methods:
- **Key escrow/backup** systems
- **Redundancy** in key management
- **Access control** mechanisms
- **Protection against DoS** attacks

### How Cryptography Helps:
- Prevents denial-of-service attacks
- Ensures authorized access to encrypted data
- Key recovery mechanisms
- Load balancing with cryptographic authentication

### Note:
Cryptography indirectly supports availability but primarily focuses on confidentiality and integrity. Availability often requires additional infrastructure (redundancy, backups, DDoS protection).

### Example 1: Key Backup/Escrow

In [ ]:
print("AVAILABILITY EXAMPLE 1: Key Backup System")
print("=" * 70)
print("Use Case: Enterprise encryption key management\n")

# Master encryption key
master_key = Fernet.generate_key()
print(f"Master key: {master_key.decode()}")

# Encrypt sensitive data
cipher = Fernet(master_key)
data = b"Critical business data"
encrypted_data = cipher.encrypt(data)

print(f"\nEncrypted data: {encrypted_data[:50]}...")

# Key escrow: Split key across multiple secure locations
key_parts = [master_key[:22], master_key[22:]]

print(f"\nKey split into {len(key_parts)} parts:")
print(f"  Part 1 (HSM-1): {key_parts[0]}")
print(f"  Part 2 (HSM-2): {key_parts[1]}")

# Simulate key recovery
print("\nScenario: Primary key lost, recovering from escrow...")
recovered_key = key_parts[0] + key_parts[1]

cipher2 = Fernet(recovered_key)
recovered_data = cipher2.decrypt(encrypted_data)

print("✓ Key recovered from escrow")
print(f"✓ Data decrypted: {recovered_data.decode()}")
print("\n✓ Availability maintained through key backup")
print("✓ No data loss even if primary key lost")

### Example 2: Rate Limiting with Cryptographic Tokens

In [ ]:
print("AVAILABILITY EXAMPLE 2: API Rate Limiting")
print("=" * 70)
print("Use Case: Protecting services from abuse\n")


# Server secret for token generation
server_secret = b"secret_key_for_rate_limiting"


def generate_token(user_id, timestamp):
    """Generate cryptographic rate limit token"""
    message = f"{user_id}:{timestamp}".encode()
    token = hmac_lib.new(server_secret, message, hashlib.sha256).hexdigest()
    return token


def verify_token(user_id, timestamp, token):
    """Verify rate limit token"""
    expected = generate_token(user_id, timestamp)
    return hmac_lib.compare_digest(token, expected)


# Legitimate user
user_id = "alice"
timestamp = int(time.time())
token = generate_token(user_id, timestamp)

print(f"User: {user_id}")
print(f"Token: {token[:40]}...")

if verify_token(user_id, timestamp, token):
    print("\n✓ Token valid - Request allowed")
    print("✓ Legitimate users can access service")

# Attacker tries to forge token
fake_token = "forged_token_12345" + "0" * 24
if not verify_token(user_id, timestamp, fake_token):
    print("\n✗ Invalid token - Request blocked")
    print("✓ DDoS/abuse attempts rejected")
    print("✓ Service availability protected")

### Example 3: Multi-Factor Authentication (MFA)

In [ ]:
print("AVAILABILITY EXAMPLE 3: Account Recovery with MFA")
print("=" * 70)
print("Use Case: Preventing account lockouts\n")


# User sets up backup codes (cryptographically random)
backup_codes = [secrets.token_hex(4) for _ in range(5)]

print("Backup codes generated (store securely):")
for i, code in enumerate(backup_codes, 1):
    print(f"  {i}. {code}")

# Hash codes for storage
stored_hashes = [hashlib.sha256(code.encode()).hexdigest() for code in backup_codes]

print("\nScenario: User lost phone (MFA device)...")
print("User enters backup code for recovery...\n")

# User uses backup code
user_code = backup_codes[0]
user_hash = hashlib.sha256(user_code.encode()).hexdigest()

if user_hash in stored_hashes:
    print(f"✓ Backup code '{user_code}' verified")
    print("✓ Access granted")
    print("✓ User can set up new MFA device")
    print("\n✓ Availability maintained through backup authentication")
    print("✓ Account not permanently locked out")

    # Mark code as used
    stored_hashes.remove(user_hash)
    print(f"\nRemaining backup codes: {len(stored_hashes)}")
else:
    print("✗ Invalid backup code")

## 5. CIA Triad in Practice - Complete Example

In [ ]:
print("COMPLETE CIA TRIAD EXAMPLE: Secure Email System")
print("=" * 70)

# Setup
sender_private = rsa.generate_private_key(65537, 2048, default_backend())
sender_public = sender_private.public_key()
recipient_private = rsa.generate_private_key(65537, 2048, default_backend())
recipient_public = recipient_private.public_key()

email_message = b"Confidential business proposal - Q4 budget: $1M"

print("\n1. CONFIDENTIALITY - Encryption")
print("-" * 70)

# Encrypt with recipient's public key
encrypted_email = recipient_public.encrypt(
    email_message,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None
    ),
)
print("✓ Email encrypted with recipient's public key")
print("✓ Only recipient can read (has private key)")
print(f"Encrypted: {encrypted_email[:50].hex()}...")

print("\n2. INTEGRITY - Digital Signature")
print("-" * 70)

# Sign with sender's private key
signature = sender_private.sign(
    email_message,
    padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
    hashes.SHA256(),
)
print("✓ Email signed with sender's private key")
print("✓ Recipient can verify authenticity")
print("✓ Tampering detection enabled")
print(f"Signature: {signature[:50].hex()}...")

print("\n3. AVAILABILITY - Key Backup")
print("-" * 70)

# Export keys for backup
key_pem = recipient_private.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.BestAvailableEncryption(b"backup_password"),
)
print("✓ Private key backed up (encrypted with password)")
print("✓ Can recover access if primary device lost")
print("✓ Key stored in secure vault/HSM")

print("\n4. RECIPIENT RECEIVES EMAIL")
print("-" * 70)

# Decrypt
decrypted = recipient_private.decrypt(
    encrypted_email,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None
    ),
)

# Verify signature
try:
    sender_public.verify(
        signature,
        decrypted,
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256(),
    )
    signature_valid = True
except Exception:
    signature_valid = False

print(f"Decrypted: {decrypted.decode()}")
print("\n✓ Confidentiality: Message was encrypted")
print(f"✓ Integrity: Signature valid = {signature_valid}")
print("✓ Availability: Backup allows key recovery")
print("\n✓ ALL THREE CIA PRINCIPLES ACHIEVED!")

## Summary

### CIA Triad Overview:

| Principle | Cryptographic Methods | Practical Examples |
|-----------|----------------------|--------------------|
| **Confidentiality** | • Symmetric encryption (AES)<br>• Asymmetric encryption (RSA)<br>• TLS/SSL | 1. File/disk encryption<br>2. HTTPS for web traffic<br>3. Database column encryption |
| **Integrity** | • Hash functions (SHA-256)<br>• Digital signatures (RSA)<br>• HMAC | 1. File checksum verification<br>2. Code signing<br>3. API message authentication |
| **Availability** | • Key backup/escrow<br>• Rate limiting tokens<br>• Multi-factor authentication | 1. Key recovery systems<br>2. DDoS protection<br>3. Backup authentication codes |

### How Cryptography Achieves CIA:

**Confidentiality:**
- Encryption makes data unreadable without key
- Only authorized parties with keys can access
- Protects data at rest and in transit

**Integrity:**
- Hash functions detect any modification
- Digital signatures prove authenticity
- One-way functions prevent forgery

**Availability:**
- Key backup prevents data loss
- Cryptographic tokens prevent abuse
- MFA provides alternative access methods

### Real-World Application:
Most secure systems use **all three principles together**:
- Online banking: Encryption (C) + signatures (I) + MFA (A)
- Cloud storage: Encryption (C) + checksums (I) + key recovery (A)
- Email (PGP/S/MIME): Encryption (C) + signing (I) + key backup (A)

### Best Practices:
- Don't prioritize one principle at expense of others
- Use established cryptographic libraries
- Implement defense in depth
- Regular security audits
- Key management is critical for all three